<a href="https://colab.research.google.com/github/factorLee/FastCampus/blob/main/EL_Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble Learning_Bagging

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# 현재경로 확인
os.getcwd()

'/content'

In [6]:
# kc_house 데이터 불러오기
data=pd.read_csv("./kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [7]:
# 집값의 가격을 예측하는 문제
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [8]:
ncar=data.shape[0]
nvar=data.shape[1]
print(ncar) # 데이터 개수
print(nvar) # 변수 개수

21613
14


# 의미가 없다고 판단되는 변수 제거

In [9]:
data = data.drop(['id','date','zipcode','lat','long'], axis=1)

# 범주형 변수를 이진형 변수로 변형
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0,1 로 표현한다.
- 데이터에서 0,1로 표현되어 있으므로 과정 생략

# 설명변수와 타겟변수를 분리, 학습데이터와 평과데이터 분리

In [10]:
# 설명변수와 타겟변수 분리
feature_columns = list(data.columns.difference(['price'])) # 'price'를 빼고 나머지를 feature_columns에 넣는다.
X = data[feature_columns]
y = data['price']
# 학습데이터와 평과데이터 분리
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size=0.3,random_state=42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(15129, 8) (6484, 8) (15129,) (6484,)


# 학습데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (State_Models)


In [11]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant='add')
sm_model = sm.OLS(train_y, sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Sun, 21 Mar 2021   Prob (F-statistic):               0.00
Time:                        13:59:57   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
sm_test_x=sm.add_constant(test_x, has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)

In [13]:
sqrt(mean_squared_error(sm_model_predict, test_y)) # RMSE

239804.2967085816

# Bagging 한 결과가 일반적인 결과보다 좋은지 확인

In [15]:
# for문을 이용한 앙상블기법(라이브러리를 다 쓰지 않음)
import random
bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 1부터 train 개수의 인덱스를 가져옴
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터 복원 추출
    sm_test_x = train_x.iloc[random_data_index,] # random_data_index 인덱스를 가져온다.
    sm_test_y = train_y.iloc[random_data_index,]
    sm.add_constant(sm_train_x, has_constant="add") # 상수항 추가

# 2021.03.21